# Computer Vision - Semester 8, Spring 2021

## Laboratory Project 1: Points of interest detection & feature extraction in images
     Christos Dimopoulos (031 17 037)
     Dimitris Dimos (031 17 165)

## Part 3: Image Matching & Classification

In [6]:
import importlib
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
from matplotlib import rcParams
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist

import cv21_lab1_part3_utils as p3
from ipynb.fs.full.part2 import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
# some typical values for the following parameters
typical = {
    'sigma': 2,
    'rho': 2.5,
    'k': 0.05,
    'theta_corn': 0.005,
    's': 1.5,
    'N': 4
}

## 3.1. Image Matching after Rotation & Scaling

In [7]:
# SURF Descriptor
desc_fun1 = lambda I, kp: p3.featuresSURF(I,kp)

# HOG Descriptor
desc_fun2 = lambda I, kp: p3.featuresHOG(I,kp)

(A) Firstly, we evaluate matching for the Harris-Stephend Edge Detector using both SURF & HOG Methods.

In [35]:
# Here is a lambda which acts as a wrapper for detector function, e.g. harrisDetector.
# The detector arguments are, in order: image, sigma, rho, k, threshold.
detect_fun1 = lambda I: HarrisStephens(I, typical['sigma'],
                                    typical['rho'],
                                    typical['k'],
                                    typical['theta_corn'])

In [17]:
# Matching Evaluation for Edge Detection using Harris-Stephens Algorithm

# Execute evaluation by providing the above functions as arguments
# Returns 2 1x3 arrays containing the errors
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun1, desc_fun1)
print("Harris Stephens Edge Detection - SURF:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Harris Stephens Edge Detection - SURF:
Avg. Scale Error for Image 1: 0.003
Avg. Theta Error for Image 1: 1.968
Avg. Scale Error for Image 2: 0.002
Avg. Theta Error for Image 2: 0.318
Avg. Scale Error for Image 3: 0.097
Avg. Theta Error for Image 3: 12.909


In [18]:
# Execute evaluation by providing the above functions as arguments
# Returns 2 1x3 arrays containing the errors
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun1, desc_fun2)
print("Harris Stephens Edge Detection - HOG:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Harris Stephens Edge Detection - HOG:
Avg. Scale Error for Image 1: 0.186
Avg. Theta Error for Image 1: 22.619
Avg. Scale Error for Image 2: 0.351
Avg. Theta Error for Image 2: 19.199
Avg. Scale Error for Image 3: 0.285
Avg. Theta Error for Image 3: 23.699


(B) Next up, we evaluate matching with Harris-Laplace Multiscale Edge Detectors, using both SURF & HOG Methods.

In [36]:
detect_fun2 = lambda I: HarrisLaplacian(I,
                                    typical['sigma'],
                                    typical['rho'],
                                    typical['k'],
                                    typical['theta_corn'],
                                    typical['s'],
                                    typical['N'])

In [42]:
# SURF Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun2, desc_fun1)
print("Harris Laplace Multiscale Edge Detection - SURF:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Harris Laplace Multiscale Edge Detection - SURF:
Avg. Scale Error for Image 1: 0.001
Avg. Theta Error for Image 1: 0.077
Avg. Scale Error for Image 2: 0.002
Avg. Theta Error for Image 2: 0.183
Avg. Scale Error for Image 3: 0.002
Avg. Theta Error for Image 3: 0.144


In [43]:
# HOG Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun2, desc_fun2)
print("Harris Laplace Multiscale Edge Detection - HOG:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Harris Laplace Multiscale Edge Detection - HOG:
Avg. Scale Error for Image 1: 0.151
Avg. Theta Error for Image 1: 22.059
Avg. Scale Error for Image 2: 0.214
Avg. Theta Error for Image 2: 9.729
Avg. Scale Error for Image 3: 0.298
Avg. Theta Error for Image 3: 23.137


(C) Following, we evaluate Matching of Blob Detector, using both SURF and HOG

In [20]:
detect_fun3 = lambda I: BlobDetect(I,
              typical['sigma'],
              typical['theta_corn'])

In [22]:
# SURF Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun3, desc_fun1)
print("Blob Detection - SURF:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Blob Detection - SURF:
Avg. Scale Error for Image 1: 0.013
Avg. Theta Error for Image 1: 1.302
Avg. Scale Error for Image 2: 0.002
Avg. Theta Error for Image 2: 0.142
Avg. Scale Error for Image 3: 0.001
Avg. Theta Error for Image 3: 0.144


In [23]:
# HOG Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun3, desc_fun2)
print("Blob Detection - HOG:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Blob Detection - HOG:
Avg. Scale Error for Image 1: 0.119
Avg. Theta Error for Image 1: 13.276
Avg. Scale Error for Image 2: 0.121
Avg. Theta Error for Image 2: 22.672
Avg. Scale Error for Image 3: 0.145
Avg. Theta Error for Image 3: 19.068


(D) Following, we evaluate Matching of Multiscale Blob Detector, using both SURF and HOG

In [9]:
detect_fun4 = lambda I: HessianLaplacian(I,
                            typical['sigma'],
                            typical['theta_corn'],
                            typical['s'],
                            typical['N'])

In [10]:
# SURF Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun4, desc_fun1)
print("Hessian - Laplacian Multiscale Blob Detection - SURF:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Hessian - Laplacian Multiscale Blob Detection - SURF:
Avg. Scale Error for Image 1: 0.002
Avg. Theta Error for Image 1: 0.054
Avg. Scale Error for Image 2: 0.002
Avg. Theta Error for Image 2: 0.079
Avg. Scale Error for Image 3: 0.001
Avg. Theta Error for Image 3: 0.070


In [11]:
# HOG Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun4, desc_fun2)
print("Hessian - Laplacian Multiscale Blob Detection - HOG:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Hessian - Laplacian Multiscale Blob Detection - HOG:
Avg. Scale Error for Image 1: 0.091
Avg. Theta Error for Image 1: 10.699
Avg. Scale Error for Image 2: 0.182
Avg. Theta Error for Image 2: 21.689
Avg. Scale Error for Image 3: 0.191
Avg. Theta Error for Image 3: 18.588


(E) Finally, we evaluate Matching done with Multiscale Box Filter Detectors, using both SURF and HOG descriptors

In [12]:
detect_fun5 = lambda I: multiscaleBoxFilterDetect(I,
                                           typical['sigma'],
                                           typical['theta_corn'],
                                           typical['s'],
                                           typical['N'])

In [13]:
# SURF Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun5, desc_fun1)
print("Box Filter Multiscale Blob Detection - SURF:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Box Filter Multiscale Blob Detection - SURF:
Avg. Scale Error for Image 1: 0.003
Avg. Theta Error for Image 1: 0.188
Avg. Scale Error for Image 2: 0.034
Avg. Theta Error for Image 2: 8.495
Avg. Scale Error for Image 3: 0.012
Avg. Theta Error for Image 3: 2.731


In [14]:
# HOG Descriptor
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun5, desc_fun2)
print("Box Filter Multiscale Blob Detection - HOG:")
print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
print('Avg. Scale Error for Image 2: {:.3f}'.format(avg_scale_errors[1]))
print('Avg. Theta Error for Image 2: {:.3f}'.format(avg_theta_errors[1]))
print('Avg. Scale Error for Image 3: {:.3f}'.format(avg_scale_errors[2]))
print('Avg. Theta Error for Image 3: {:.3f}'.format(avg_theta_errors[2]))

Box Filter Multiscale Blob Detection - HOG:
Avg. Scale Error for Image 1: 0.208
Avg. Theta Error for Image 1: 24.985
Avg. Scale Error for Image 2: 0.239
Avg. Theta Error for Image 2: 30.579
Avg. Scale Error for Image 3: 0.311
Avg. Theta Error for Image 3: 21.849


## 3.2. Image Classification

In [15]:
def mean_acc(features):
    accs = []
    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = p3.createTrainTest(features, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)

    return 100.0*np.mean(accs)

(A) Firstly, we evaluate image classification with Harris-Laplace Multiscale Edge Detectors, using both SURF & HOG Methods.

In [37]:
detect_fun1 = lambda I: HarrisLaplacian(I,
                                    typical['sigma'],
                                    typical['rho'],
                                    typical['k'],
                                    typical['theta_corn'],
                                    typical['s'],
                                    typical['N'])

In [47]:
# SURF Method
# Extract features from the provided dataset.
feats1a = p3.FeatureExtraction(detect_fun1, desc_fun1, saveFile = 'HarrisLaplacian_SURF')

Time for feature extraction: 333.750


In [328]:
acc1a = mean_acc(feats1a)
print('Mean accuracy for Harris-Laplace with SURF descriptors: {:.3f}%'.format(acc1a))

Mean accuracy for Harris-Laplace with SURF descriptors: 61.103%


In [66]:
# HOG Method
# Extract features from the provided dataset.
feats1b = p3.FeatureExtraction(detect_fun1, desc_fun2, saveFile = 'HarrisLaplacian_HOG')

Time for feature extraction: 347.916


In [329]:
acc1b = mean_acc(feats1b)
print('Mean accuracy for Harris-Laplace with HOG descriptors: {:.3f}%'.format(acc1b))

Mean accuracy for Harris-Laplace with HOG descriptors: 66.483%


(B) Up next, we evaluate image classification with Hessian-Laplace Multiscale Blob Detectors, using both SURF & HOG Methods.

In [38]:
detect_fun2 = lambda I: HessianLaplacian(I,
                            typical['sigma'],
                            typical['theta_corn'],
                            typical['s'],
                            typical['N'])

In [52]:
# SURF Method
# Extract features from the provided dataset.
feats2a = p3.FeatureExtraction(detect_fun2, desc_fun1, saveFile = 'HessianLaplacian_SURF')

Time for feature extraction: 328.251


In [330]:
acc2a = mean_acc(feats2a)
print('Mean accuracy for Hessian-Laplace with SURF descriptors: {:.3f}%'.format(acc2a))

Mean accuracy for Hessian-Laplace with SURF descriptors: 62.207%


In [55]:
# HOG Method
# Extract features from the provided dataset.
feats2b = p3.FeatureExtraction(detect_fun2, desc_fun2, saveFile = 'HessianLaplacian_HOG')

Time for feature extraction: 381.060


In [331]:
acc2b = mean_acc(feats2b)
print('Mean accuracy for Hessian-Laplace with HOG descriptors: {:.3f}%'.format(acc2b))

Mean accuracy for Hessian-Laplace with HOG descriptors: 65.931%


(C) Finally, we evaluate image classification with Box-Filter Multiscale Blob Detectors, using both SURF & HOG Methods.

In [39]:
detect_fun3 = lambda I: multiscaleBoxFilterDetect(I,
                                           typical['sigma'],
                                           typical['theta_corn'],
                                           typical['s'],
                                           typical['N'])

In [17]:
# SURF Method
# Extract features from the provided dataset.
feats3a = p3.FeatureExtraction(detect_fun3, desc_fun1, saveFile = 'BoxFilter_SURF')

Time for feature extraction: 327.403


In [18]:
acc3a = mean_acc(feats3a)
print('Mean accuracy for Multiscale Box-Filters with SURF descriptors: {:.3f}%'.format(acc3a))

Mean accuracy for Multiscale Box-Filters with SURF descriptors: 50.621%


In [19]:
# HOG Method
# Extract features from the provided dataset.
feats3b = p3.FeatureExtraction(detect_fun3, desc_fun2, saveFile = 'BoxFilter_HOG')

Time for feature extraction: 386.113


In [20]:
acc3b = mean_acc(feats3b)
print('Mean accuracy for Multiscale Box-Filters with HOG descriptors: {:.3f}%'.format(acc3b))

Mean accuracy for Multiscale Box-Filters with HOG descriptors: 57.793%


## BONUS: Create Bag of Visual Words using Histograms

In [21]:
def histogram_maker(image, centroids):
    #compute pairwise euclidean distance for each image from each centroid
    dist = cdist(image, centroids)
    
    # keep as label the centroid of minimum distance
    labels = np.zeros(dist.shape[0])
    for i in range(labels.shape[0]):
        labels[i] = np.argwhere(dist[i] == np.min(dist[i]))
    
    # Create Histogram
    histogram, bins = np.histogram(labels, np.arange(0,centroids.shape[0]))
    
    # Normalize
    norm = np.linalg.norm(histogram)
    histogram = histogram / norm
    
    return histogram

In [29]:
def BonusBoVW(data_train, data_test):
    # Define useful variables
    num_centroids = 500;
    subset_percent = 0.5;

    data_descriptors = np.vstack(data_train) #concatenate all of them

    # Get a random subset of data_descriptors.
    subset_len = np.ceil(np.shape(data_descriptors)[0] * subset_percent)
    i = np.random.choice(np.shape(data_descriptors)[0], int(subset_len))
    sample = data_descriptors[i]

    # Perform k-means algorithm
    kmeans = KMeans(n_clusters=num_centroids).fit(sample)
    idx, centroids = kmeans.labels_, kmeans.cluster_centers_ 
    
    # Create Bag of Words for Train Test using Histograms
    BOF_tr = []
    for image in (data_train):
        BOF_tr.append(histogram_maker(image, centroids).tolist())
    BOF_tr = np.stack(BOF_tr, axis=0)
    
    # Repeat procedure for Test Set
    BOF_ts = []
    for image in (data_test):
        BOF_ts.append(histogram_maker(image, centroids).tolist())
    BOF_ts = np.stack(BOF_ts, axis=0)
    
    return BOF_tr, BOF_ts

In [28]:
def bonus_acc(features):
    accs = []
    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = p3.createTrainTest(features, k)

        # Use our own Bof of Visual Words Function
        BOF_tr, BOF_ts = BonusBoVW(data_train, data_test)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
        
    return 100.0*np.mean(accs)

Evaluate Classification with new Accuracy Metric for Different Detectors & Descriptors

In [45]:
acc1a = bonus_acc(feats1a)
print('Mean accuracy for Harris-Laplace with SURF descriptors: {:.3f}%'.format(acc1a))

Mean accuracy for Harris-Laplace with SURF descriptors: 57.241%


In [42]:
acc1b = bonus_acc(feats1b)
print('Mean accuracy for Harris-Laplace with HOG descriptors: {:.3f}%'.format(acc1b))

Mean accuracy for Harris-Laplace with HOG descriptors: 65.655%


In [43]:
acc2a = bonus_acc(feats2a)
print('Mean accuracy for Hessian-Laplace with SURF descriptors: {:.3f}%'.format(acc2a))

Mean accuracy for Hessian-Laplace with SURF descriptors: 61.379%


In [46]:
acc2b = bonus_acc(feats2b)
print('Mean accuracy for Hessian-Laplace with HOG descriptors: {:.3f}%'.format(acc2b))

Mean accuracy for Hessian-Laplace with HOG descriptors: 69.379%


In [32]:
acc3a = bonus_acc(feats3a)
print('Mean accuracy for Multiscale Box-Filters with SURF descriptors: {:.3f}%'.format(acc3a))

Mean accuracy for Multiscale Box-Filters with SURF descriptors: 50.069%


In [31]:
acc3b = bonus_acc(feats3b)
print('Mean accuracy for Multiscale Box-Filters with HOG descriptors: {:.3f}%'.format(acc3b))

Mean accuracy for Multiscale Box-Filters with HOG descriptors: 61.379%
